In [1]:
import matplotlib
import numpy as np
import os
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm

import speclite
from astropy.cosmology import default_cosmology
from astropy.units import Quantity
from slsim.Observations.roman_speclite import configure_roman_filters, filter_names
from slsim.lens_pop import LensPop
from tqdm import tqdm

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs
import mejiro
from mejiro.helpers import survey_sim

In [2]:
# load Roman WFI filters
configure_roman_filters()
roman_filters = filter_names()
roman_filters.sort()
_ = speclite.filters.load_filters(*roman_filters[:8])
print('Configured Roman filters')

# load SkyPy config file
module_path = os.path.dirname(mejiro.__file__)
skypy_config = os.path.join(module_path, 'data', 'roman_hlwas.yml')
print(f'Loaded SkyPy configuration file {skypy_config}')

# set HLWAS parameters
survey_area = float(util.load_skypy_config(skypy_config)['fsky'][:-5])
sky_area = Quantity(value=survey_area, unit='deg2')
cosmo = default_cosmology.get()
pipeline_params = util.hydra_to_dict(config.pipeline)
bands = pipeline_params['bands']

Configured Roman filters
Loaded SkyPy configuration file /nfshome/bwedig/mejiro/mejiro/data/roman_hlwas.yml


In [3]:
# define cuts on the intrinsic deflector and source populations (in addition to the skypy config file)
survey_params = util.hydra_to_dict(config.survey)

kwargs_deflector_cut = {
    'band': 'F106',
    'band_max': survey_params['deflector_cut_band_max'],
    'z_min': 0.01,
    'z_max': 2.
}
kwargs_source_cut = {
    'band': 'F106',
    'band_max': survey_params['source_cut_band_max'],
    'z_min': 0.01,
    'z_max': 5.
}

In [4]:
# create the lens population
print('Defining galaxy population...')
lens_pop = LensPop(deflector_type="all-galaxies",
                   source_type="galaxies",
                   kwargs_deflector_cut=kwargs_deflector_cut,
                   kwargs_source_cut=kwargs_source_cut,
                   kwargs_mass2light=None,
                   skypy_config=skypy_config,
                   sky_area=sky_area,
                   cosmo=cosmo)
print('Defined galaxy population')

Defining galaxy population...


Defined galaxy population


In [5]:
# draw initial detectable lens population
print('Identifying detectable lenses...')
kwargs_lens_detectable_cut = {
    'min_image_separation': 0.2,
    'max_image_separation': 10,
    'mag_arc_limit': {'F106': 25}
}
lens_population = lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_detectable_cut)
print(f'Number of detectable lenses from first set of criteria: {len(lens_population)}')

Identifying detectable lenses...


Number of detectable lenses from first set of criteria: 30934


In [10]:
# set up dict to capture some information about which candidates got filtered out
filtered_sample = {}
filtered_sample['total'] = len(lens_population)
num_samples = 16
filter_1, filter_2 = 0, 0
filtered_sample['filter_1'] = []
filtered_sample['filter_2'] = []

# apply additional detectability criteria
limit = None
detectable_gglenses = []
for candidate in tqdm(lens_population):
    # 1. Einstein radius and Sersic radius
    _, kwargs_params = candidate.lenstronomy_kwargs(band='F106')
    lens_mag = candidate.deflector_magnitude(band='F106')

    if kwargs_params['kwargs_lens'][0]['theta_E'] < kwargs_params['kwargs_lens_light'][0][
        'R_sersic'] and lens_mag < 15:
        filter_1 += 1
        if filter_1 <= num_samples:
            filtered_sample['filter_1'].append(candidate)
        continue

    # 2. SNR
    snr, _ = survey_sim.get_snr(candidate, 'F106')

    if snr < 5:
        filter_2 += 1
        if filter_2 <= num_samples:
            filtered_sample['filter_2'].append(candidate)
        continue

    # if both criteria satisfied, consider detectable
    detectable_gglenses.append(candidate)

    # if I've imposed a limit above this loop, exit the loop
    if limit is not None and len(detectable_gglenses) == limit:
        break

print(f'{len(detectable_gglenses)} detectable lens(es)')

100%|██████████| 30934/30934 [01:53<00:00, 271.37it/s]

40 detectable lens(es)


In [15]:
util.pickle(os.path.join(data_dir, 'sample_detectable_gglenses.pkl'), detectable_gglenses)

In [16]:
sample_gglens = np.random.choice(detectable_gglenses, 1)
util.pickle(os.path.join(data_dir, 'sample_gglens.pkl'), sample_gglens)